In [537]:
PATH_TO_FILE = 'handstandAvg.obj'
PADDING = 0.10
TARGET_CHUNKS = 64
ISO_LEVEL = 1.3
TARGET_FACES = 500

In [538]:
import trimesh
import numpy as np

# Load mesh from file (e.g., OBJ, PLY, STL)
mesh = trimesh.load(PATH_TO_FILE)

# Get axis-aligned bounding box
bounds = mesh.bounds  # shape: (2, 3), [[min_x, min_y, min_z], [max_x, max_y, max_z]]
min_bound, max_bound = bounds

# Compute dimensions (width, height, depth)
dimensions = max_bound - min_bound

print("Bounding Box Min:", min_bound)
print("Bounding Box Max:", max_bound)
print("Mesh Dimensions (X, Y, Z):", dimensions)

Bounding Box Min: [-1.240334  0.616115 -0.656867]
Bounding Box Max: [-0.033728  2.343307  0.678381]
Mesh Dimensions (X, Y, Z): [1.206606 1.727192 1.335248]


In [539]:
# Determine padding: 10% of the longest side
longest_side = dimensions.max()
padding = PADDING * longest_side

# Apply padding to each side
padded_min = min_bound - padding
padded_max = max_bound + padding
padded_bounds = np.array([padded_min, padded_max])

print("Padding:\n", padding)
print("Original Bounds:\n", bounds)
print("Padded Bounds:\n", padded_bounds)

Padding:
 0.1727192
Original Bounds:
 [[-1.240334  0.616115 -0.656867]
 [-0.033728  2.343307  0.678381]]
Padded Bounds:
 [[-1.4130532  0.4433958 -0.8295862]
 [ 0.1389912  2.5160262  0.8511002]]


In [540]:
# Desired chunk count along the longest axis
target_chunks = TARGET_CHUNKS
padded_dimensions = padded_max - padded_min
voxel_size = padded_dimensions.max() / target_chunks

# Compute number of chunks per axis (rounded up)
chunk_counts = np.ceil(padded_dimensions / voxel_size).astype(int)

# Recompute max_bound to exactly fit an integer number of chunks
new_padded_max = padded_min + chunk_counts * voxel_size

# Final grid info
print("Voxel Size:", voxel_size)
print("Chunk Counts (X, Y, Z):", chunk_counts)
print("Updated Max Bound:", new_padded_max)
print("Final Bounds:\n", np.array([padded_min, new_padded_max]))

Voxel Size: 0.03238484999999999
Chunk Counts (X, Y, Z): [48 64 52]
Updated Max Bound: [0.1414196 2.5160262 0.854426 ]
Final Bounds:
 [[-1.4130532  0.4433958 -0.8295862]
 [ 0.1414196  2.5160262  0.854426 ]]


In [541]:
# Chunk counts per axis
X, Y, Z = chunk_counts
half_voxel = voxel_size * 0.5

# min_bound and voxel_size are from earlier steps
y_coords = np.linspace(
    padded_min[1] + half_voxel,  # start at first voxel center
    new_padded_max[1] - half_voxel,  # last center
    Y  # how many values
)

z_coords = np.linspace(
    padded_min[2] + half_voxel,
    new_padded_max[2] - half_voxel,
    Z
)

print("Sample y_coords:", y_coords[:5])
print("Sample z_coords:", z_coords[:5])


Sample y_coords: [0.45958823 0.49197308 0.52435793 0.55674278 0.58912763]
Sample z_coords: [-0.81339377 -0.78100892 -0.74862407 -0.71623923 -0.68385438]


In [542]:
# 2. Create Y-Z grid
yy, zz = np.meshgrid(y_coords, z_coords, indexing='ij')

# 3. Fill X with constant value before the mesh
origin_x = padded_min[0] - voxel_size
xx = np.full_like(yy, origin_x)

# 4. Combine into (Y, Z, 3) array of ray origins
ray_grid = np.stack([xx, yy, zz], axis=-1)

# 5. Flatten to (N, 3) array
ray_origins = ray_grid.reshape(-1, 3)

# 6. Create matching (N, 3) ray directions
ray_directions = np.tile([1.0, 0.0, 0.0], (ray_origins.shape[0], 1))

print("Ray origins shape:", ray_origins.shape)
print("Ray directions shape:", ray_directions.shape)

Ray origins shape: (3328, 3)
Ray directions shape: (3328, 3)


In [543]:
# Convert ray origins into a point cloud mesh (just vertices, no faces)
points_as_mesh = trimesh.points.PointCloud(ray_origins)

# Combine the mesh and the point cloud into a scene
scene = trimesh.Scene()
scene.add_geometry(mesh, node_name="original_mesh")
scene.add_geometry(points_as_mesh, node_name="ray_origins")

# Export to OBJ (OBJ supports vertex-only point clouds)
combined_obj = scene.export(file_type='obj')

# Save to disk
with open("mesh_with_rays.obj", "w") as f:
    f.write(combined_obj)

print("Exported to mesh_with_rays.obj")

Exported to mesh_with_rays.obj


In [544]:
# Use Trimesh's ray intersector (it will fall back to pure Python if needed)
ray_intersector = mesh.ray

# Perform ray-mesh intersection test
locations, index_ray, index_tri = ray_intersector.intersects_location(
    ray_origins,
    ray_directions,
    multiple_hits=True
)

print("Number of intersections:", len(locations))

Number of intersections: 1350


In [545]:
# Count how many times each ray hit the mesh
num_rays = len(ray_origins)
hit_counts = np.bincount(index_ray, minlength=num_rays)

# Mark rays that intersected the mesh as occupied
inside_flags = (hit_counts > 0)

# Reshape to 2D Y x Z grid
occupancy_yz = inside_flags.reshape((Y, Z))

print("Occupancy mask shape:", occupancy_yz.shape)

Occupancy mask shape: (64, 52)


In [546]:
# 2. Get indices where occupancy is True
iy, iz = np.where(occupancy_yz)  # index arrays

# 3. Convert indices to coordinates
inside_points = np.column_stack([
    np.full_like(iy, origin_x, dtype=float),  # fixed X
    y_coords[iy],
    z_coords[iz]
])

print(f"Found {len(inside_points)} occupied (Y,Z) rays")

Found 650 occupied (Y,Z) rays


In [547]:
# Create a point cloud from the occupancy points
occupancy_pc = trimesh.points.PointCloud(inside_points)

# Combine the mesh and the point cloud into a scene
scene = trimesh.Scene()
scene.add_geometry(mesh, node_name="original_mesh")
scene.add_geometry(occupancy_pc, node_name="inside_points")

# Export to OBJ (OBJ supports vertex-only point clouds)
combined_obj = scene.export(file_type='obj')

# Save to disk
with open("occupancy_yz.obj", "w") as f:
    f.write(combined_obj)

print("Exported occupancy points to occupancy_yz.obj")

Exported occupancy points to occupancy_yz.obj


In [548]:
occupancy_grid = np.zeros((X, Y, Z), dtype=bool)

# Map ray index → X hit positions
from collections import defaultdict

ray_hits = defaultdict(list)
for loc, ray_idx in zip(locations, index_ray):
    ray_hits[ray_idx].append(loc[0])  # X coordinate

# Loop over YZ grid
for y in range(Y):
    for z in range(Z):
        ray_idx = y * Z + z 
        hits = ray_hits.get(ray_idx, [])

        if len(hits) < 2:
            continue  # not enough hits to fill between

        # Sort the x-values to ensure they are in ascending order
        hits.sort()

        for i in range(0, len(hits) - 1, 2):
            x0, x1 = hits[i], hits[i + 1]

            # Convert world X to grid index
            gx0 = int(np.ceil((min(x0, x1) - padded_min[0]) / voxel_size))
            gx1 = int(np.floor((max(x0, x1) - padded_min[0]) / voxel_size))

            if gx0 > gx1:
                continue  # skip invalid ranges

            occupancy_grid[gx0:gx1 + 1, y, z] = True

In [549]:
occupied = np.argwhere(occupancy_grid)  # shape (N, 3), with indices [x, y, z]
pc = trimesh.points.PointCloud(occupied)

# Export to OBJ (OBJ supports vertex-only point clouds)
combined_obj = pc.export(file_type='obj')

# Save to disk
with open("occupancy_xyz.obj", "w") as f:
    f.write(combined_obj)

print("Exported 3D occupancy points to occupancy_xyz.obj")

Exported 3D occupancy points to occupancy_xyz.obj


In [550]:
import numpy as np
from scipy.ndimage import distance_transform_edt

# occupancy: shape (X, Y, Z), dtype=bool, where True = inside

# Distance to the nearest "outside" voxel (from inside)
dist_inside = distance_transform_edt(occupancy_grid)

# Distance to the nearest "inside" voxel (from outside)
dist_outside = distance_transform_edt(~occupancy_grid)

# Signed distance field: negative inside, positive outside
sdf = dist_outside - dist_inside

In [551]:
from scipy.ndimage import gaussian_filter
#sdf = gaussian_filter(sdf, sigma=0.5)

In [552]:
from skimage import measure

verts, faces, normals, values = measure.marching_cubes(
    sdf, level=ISO_LEVEL, spacing=(voxel_size, voxel_size, voxel_size)
)
verts += padded_min  # Shift back to world coordinates if needed

In [ ]:
import os

# --- 2. Create a Trimesh object ---
mesh = trimesh.Trimesh(vertices=verts, faces=faces, vertex_normals=normals, process=False)

# --- 3. Clean mesh ---
#mesh.remove_degenerate_faces()
#mesh.remove_duplicate_faces()
#mesh.remove_unreferenced_vertices()
#mesh.remove_infinite_values()
mesh.process(validate=True)

# Optional: reorient faces consistently
#mesh.fix_normals()

# --- 4. Visualize ---
#mesh.show()
mesh.export('reconstructed.obj')
#os.startfile('reconstructed.obj')


'# https://github.com/mikedh/trimesh\nv -1.26084441 1.99786860 0.04480475\nv -1.25112895 1.97441348 0.04480475\nv -1.25112895 1.99786860 0.02134956\nv -1.25112895 1.99786860 0.06825994\nv -1.26084441 2.03025345 0.04480475\nv -1.25112895 2.03025345 0.02134956\nv -1.25112895 2.03025345 0.06825994\nv -1.26084441 2.06263830 0.04480475\nv -1.25112895 2.06263830 0.02134956\nv -1.25112895 2.06263830 0.06825994\nv -1.25112895 2.08609342 0.04480475\nv -1.25112895 2.09502315 0.05373441\nv -1.26084441 2.09502315 0.07718960\nv -1.25112895 2.07156803 0.07718960\nv -1.25112895 2.09502315 0.10064479\nv -1.25112895 2.11847827 0.07718960\nv -1.23201566 1.96548375 0.01241990\nv -1.21874410 1.94202863 0.01241990\nv -1.21874410 1.96548375 -0.01103529\nv -1.24219929 1.96548375 0.04480475\nv -1.21874410 1.95221214 0.04480475\nv -1.21874410 1.96548375 0.06825994\nv -1.24219929 1.99786860 0.01241990\nv -1.21874410 1.99786860 -0.01103529\nv -1.24219929 1.99786860 0.07718960\nv -1.21874410 1.97441348 0.07718960

In [554]:
print("Max SDF value:", np.max(sdf))

Max SDF value: 39.02563260217571


In [555]:
import pymeshlab

# Convert to PyMeshLab mesh
ml_mesh = pymeshlab.Mesh(mesh.vertices, mesh.faces)

# Create a MeshSet and add the mesh
ms = pymeshlab.MeshSet()
ms.add_mesh(ml_mesh, "original")

# Apply simplification
ms.apply_filter('meshing_decimation_quadric_edge_collapse', targetfacenum=TARGET_FACES)

# Get the simplified mesh
simplified = ms.current_mesh()

# Convert back to Trimesh
simplified_trimesh = trimesh.Trimesh(
    vertices=simplified.vertex_matrix(),
    faces=simplified.face_matrix()
)

simplified_trimesh.export('simplified.obj')
os.startfile('simplified.obj')